# Laboratorio 7.2

### 2- Similitud de coseno

## P4- Similitud de coseno con el Indice Invertido

### 1- Estructura del índice invertido en Python:

In [7]:
"""


El indice esta estructurado de la siguiente forma, contiene el idf para cada termino y su posting list. 
La posting list contiene el id del documento y la frecuencia del termino en el documento.

index.txt :
w1 : idf_w1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],
w2 : idf_w2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],
w3 : idf_w3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],

length ={
doc1: norm_doc1,
doc2: norm_doc2,
doc3: norm_doc3,
...
}
"""

'\nindex = {\nw1 : [(doc1, tf_w1_doc1), (doc3, tf_w1_doc3),(doc4, tf_w1_doc4),(doc10, tf_w1_doc10)],\nw2 : [(doc1, tf_w2_doc1 ), (doc2, tf_w2_doc2)],\nw3 : [(doc2, tf_w3_doc2), (doc3, tf_w3_doc3),(doc7, tf_w3_doc7)],\n}\n\nidf = {\nw1 : idf_w1,\nw2 : idf_w2,\nw3 : idf_w3,\n}\n\nlength ={\ndoc1: norm_doc1,\ndoc2: norm_doc2,\ndoc3: norm_doc3,\n...\n}\n'

# SPIMI 

In [46]:
import pickle
import csv
import nltk
import re
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import os
import sys
import heapq
import numpy as np
import pandas as pd

nltk.download('punkt')

class InvertIndexSPIMI:
    def __init__(self):
      
        self.index_main = {} # dictionary to store the position of each term in the index file
        self.length_main = {} # dictionary to store the norm for each document
        self.stemmer = SnowballStemmer('english')
        self.stoplist = []
        self.preprocessed_words_file = "preprocessed_words.txt"
        self.spimi_file = "spimi_blocks.txt"
        self.id_block = 0
        self.free_memory_available = 500000  # 0.5 MB
        self.current_line_preprocessed_words_file = 0
        self.end_preprocessed_words_file = 0
        self.min_heap_size = 500
        self.index = "index.txt"
        self.cant_documents = 18454
        self.norma_docs_file = "norma_docs.pkl"
        self.position_terms_file = "position_terms.pkl"
        self.original_file = r".\Postgres\spotify_songs.csv"
        #fd
 # TODO Process the text to create many blocks using the SPIMI algorithm
    def load_stop_list(self, filename="english_stoplist.txt"):
        with open(filename, encoding="utf-8") as file:
            self.stoplist = [line.rstrip().lower() for line in file]

    def preprocesamiento_aux(self, texto):
        words = []
        # tokenizar
        texto_tok = texto.lower()
        texto_tok = re.sub(r'[^a-zA-Z_À-ÿ]', ' ', texto)
        words = nltk.word_tokenize(texto_tok, language='english')
        # filtrar stopwords
        words = [word for word in words if word not in self.stoplist]
        # reducir palabras
        words = [self.stemmer.stem(word) for word in words]
        return words
    def get_list_tf(self, tokens, id):
        # compute the term frequency for each word in the document with id
        tf = {}
        for word in tokens:
            if word not in tf:
                tf[word] = 1
            else:
                tf[word] += 1
        return [(id, word, tf[word]) for word in tf]
    # process the text to create the preprocessed words file
    def preprocesamiento(self, pathOfFile):
        print("Preprocessing songs")      
        self.original_file = pathOfFile
        # List of tokens for each song
        preprocessed_song = []           
        with open(pathOfFile, newline='', encoding='utf-8') as file, open(self.preprocessed_words_file, 'w') as preprocessed_words:
            songs_file = csv.reader(file, delimiter=',', quotechar='"')
            # counter used as id for each song
            counter_rows = 0
            for row in songs_file:
                # skip the header
                if counter_rows == 0:
                    counter_rows += 1
                    continue
                # track_name , artist_name, lyrics
                song_details = row[1] + " " + row[2] + " " + row[3]
                # get list of tokens for each song
                preprocessed_song = self.preprocesamiento_aux(song_details)
                # get the tf for each word in the song
                list_tf_song = self.get_list_tf(preprocessed_song, counter_rows)
                counter_rows += 1
                # write the relative_id_song, word, tf in each line of file
                for tf in list_tf_song:
                    preprocessed_words.write(str(tf) + "\n")
          
    def strToList(self, string):
        string = string[:-1] 
        string = string[1:-1]
        tup = string.split(", ")
        return [int(tup[0]), tup[1], int(tup[2])]
    
    # receive the token list which is a list of tuples (id, word, tf)
    def spimi_invert(self):
        blocks_directory = r".\blocks"
        if not os.path.exists(blocks_directory):
            os.makedirs(blocks_directory)
        output_file = r".\blocks\block_" + str(self.id_block) + ".txt"
        dictionary = {}
        with open(self.preprocessed_words_file, 'r') as preprocessed_words, open(output_file, 'w') as block:
            print("In block:", self.id_block)
            preprocessed_words.seek(self.current_line_preprocessed_words_file)
            while sys.getsizeof(dictionary) < self.free_memory_available:
                # read the next token
                token = preprocessed_words.readline()
                if not token:
                    self.end_preprocessed_words_file = 1
                    break
                # convert the string to a list of relative_id_song, word and its tf
                token = self.strToList(token)
                if token[1] not in dictionary:
                    dictionary[token[1]] = [(token[0], token[2])]
                else:
                    dictionary[token[1]].append((token[0], token[2]))
            self.current_line_preprocessed_words_file = preprocessed_words.tell()
            sorted_terms = sorted(dictionary.keys())
            for term in sorted_terms:
                block.write(term + ":" + str(dictionary[term]) + "\n")
        self.id_block += 1
        
        
    def create_spimi_blocks(self): 
        print("Creating SPIMI blocks")
        # create a directory to store the blocks
        blocks_directory = r".\blocks"
        if not os.path.exists(blocks_directory):
            os.makedirs(blocks_directory)
        # read the preprocessed words file as it has the tf for each token in each song, then created size 
        # limit blocks with spimi algorithm 
        while(not self.end_preprocessed_words_file):  
            self.spimi_invert()
       
   
    """
    Para el merge se abriran todos los bloques simultaneamente, se leera la primera palabras de cada bloque,
    luego usando una pritority queue se obtendra el menor termino lexicografico y se combinaran todas las 
    listas existentes para dicho termino. Se escribira en un nuevo archivo y se repetira el proceso hasta 
    que no haya mas.
    Dado que se el minHeap se inicializa con 1 termino de cada bloque y cada vez que se hace pop, se pushea 
    un term del mismo bloque, se garantiza que el heap tiene un term de cada bloque a menos que se haya llegado
    al final de un bloque. Ademas, se itera hasta que el heap este vacio, por lo que se garantiza el 
    recorrido total por todos los bloques.
    """
   
    def merge_spimi_blocks(self):
        print("Merging SPIMI blocks")
        # get all the block names
        blocks_name = os.listdir(r".\blocks")
        # open all the blocks
        opened_blocks = []
        for block_name in blocks_name:
            opened_blocks.append(open(".\\blocks\\" + block_name, 'r'))
        # min heap for terms
        terms_from_all_blocks = []
        # dictionary to store the norm for each document
        temp_norm_dict = {}
        # dictionary to store the position of each term in the index file
        position_terms = {}
        # read the first term from each block
        counter_opened_block = 0
        for opened_block in opened_blocks:
            term = opened_block.readline()
            if term:
                term = term.split(":")
                # push (term, [...], block_number) to the heap
                heapq.heappush(terms_from_all_blocks, (term[0], term[1], counter_opened_block))
            counter_opened_block += 1
        # merge the blocks in index.txt
        with open(self.index, 'w') as index:
            prev_top_term = None
            temp_prev_top_term = []
            
            # iterate until the heap is empty
            while len(terms_from_all_blocks) > 0:
                # obtenemos el menor termino lexicograficamente 
                current_top_term = heapq.heappop(terms_from_all_blocks)
                # if the previous term is the same as the current term, merge their posting lists
                if prev_top_term and prev_top_term[0] == current_top_term[0]:
                    # convert the string to a list of tuples if necessary
                    if isinstance(prev_top_term[1], str):                
                        temp_prev_top_term = eval(prev_top_term[1])
                    # posting list of current term is always a str
                    temp_current_top_term = eval(current_top_term[1])
                    # merge posting lists
                    temp_prev_top_term.extend(temp_current_top_term)
                    # update the posting list and block number 
                    prev_top_term =(prev_top_term[0], temp_prev_top_term, current_top_term[2])
                    
                else:
                    prev_top_term = current_top_term
                    # transform to List if posting list is a str
                    if isinstance(current_top_term[1], str):
                        temp_posting_list = eval(current_top_term[1])
                        current_top_term = (current_top_term[0], temp_posting_list, current_top_term[2])
                    # compute the idf
                    idf_t = np.log10(self.cant_documents / len(current_top_term[1]))
                    # compute the tf_idf squared weights of each term for each document, doc is a tuple (doc_id, tf)
                    for doc in current_top_term[1]:
                        if doc[0] not in temp_norm_dict:
                            temp_norm_dict[doc[0]] = ((1 + np.log10(doc[1])) * idf_t) ** 2
                        else:
                            temp_norm_dict[doc[0]] += ((1 + np.log10(doc[1])) * idf_t) ** 2
                    
                    # store the position of the term in the index file
                    position_terms[current_top_term[0][1:-1]] = index.tell()
                    # write the term to the index -> term:idf:posting_list 
                    index.write(current_top_term[0] + ":" + str(idf_t) + ":" + str(current_top_term[1]) + "\n")
                    
                # read the next term from the block of the current term
                next_term_same_block = opened_blocks[current_top_term[2]].readline()
                if next_term_same_block:
                       next_term_same_block = next_term_same_block.split(":")
                       # push the next term to the heap
                       heapq.heappush(terms_from_all_blocks, (next_term_same_block[0], next_term_same_block[1], current_top_term[2]))
        
        # compute the norm of each document
        for doc in temp_norm_dict:
            temp_norm_dict[doc] = np.sqrt(temp_norm_dict[doc])
        print("Blocks merged")
        # write norms to disk with pickle to recover them later in dictionary format
        with open(self.norma_docs_file, 'wb') as file:
            pickle.dump(temp_norm_dict, file)
        # write the position of each term in the index file to disk with pickle to recover them later
        with open(self.position_terms_file, 'wb') as file:
            pickle.dump(position_terms, file)
        # close blocks
        for opened_block in opened_blocks:
            opened_block.close()
        print("Index created")
        # TODO (OPTIONAL): remove all the blocks from disk. I'm not removing because reading the blocks from disk
        # is faster than processing the text again
        
    
            
            
    def build (self, pathOfFile):
        # FIXME: Only build the process if the index file does not exist for development purposes
        if os.path.exists(self.index):
            return
        self.load_stop_list()
        self.preprocesamiento(pathOfFile)
        self.create_spimi_blocks()
        self.merge_spimi_blocks()
    
    def load_index(self):
        # load index (position of terms in index file) and the norm of documents from disk       
        try:
            with open(self.position_terms_file, 'rb') as file:
                self.index_main = pickle.load(file)
                print("Index main ready")
            with open(self.norma_docs_file, 'rb') as file:
                self.length_main = pickle.load(file)
                print("Length main ready")
        except FileNotFoundError:
            print("Index not found")

        
    def retrieval(self, query, k):
        # load pos terms and norm docs
        self.load_index()
        # diccionario para el score
        score = {}
        # preprocesar la query: extraer los terminos unicos
        post_query = self.preprocesamiento_aux(query)
        # calcular el tf-idf del query
        query_words = set(post_query)
        query_tfidf = []
        with open(self.index, 'r') as idx_file:
            for wordq in query_words:
                idfq = 0
                if wordq in self.index_main:
                    # posicion del termino en el index 
                    pos_wordq = self.index_main[wordq]
                    # read the term from the file
                    idx_file.seek(pos_wordq)
                    term_info = idx_file.readline().split(":")
                    # idf del termino
                    idfq = float(term_info[1])
              
                  
                # tf.idf del termino en el query 
                query_tfidf.append((1 + np.log10(post_query.count(wordq))) * idfq)
        # compute the norm of the query 
       
        query_norm = np.sqrt(sum([q ** 2 for q in query_tfidf]))
        # normalizar la query
       
        query_tfidf = [q / query_norm for q in query_tfidf]
        print("Query tfidf mas:", max(query_tfidf))
        query_words = list(query_words)


        temp__ = {}
        # aplicar similitud de coseno y guardarlo en el diccionario score
        with open(self.index, 'r') as idx_file:
            for word in query_words:
                if word in self.index_main:
                    pos_word = self.index_main[word]
                    idx_file.seek(pos_word)
                    term_info = idx_file.readline().split(":")
                    idf = float(term_info[1])
                    posting_list_ = eval(term_info[2])
               
                    for doc, tf in posting_list_:
                        if doc not in score:
                            score[doc] = 0
                            temp__[doc] = 0
                        # tf.idf del termino en el documento por el tf.idf del termino en el query
                        tf_idf_ = (1 + np.log10(tf)) * idf
                        score[doc] += query_tfidf[query_words.index(word)] * tf_idf_
                        temp__[doc] +=(tf*idf)
            
            print("max1: ", max(temp__))
            for d in score:
                # normalizar el score
                score[d] = score[d] / self.length_main[d]
                temp__[d] = temp__[d] / self.length_main[d]
                print(temp__[d], " ", self.length_main[d])
            print("max: ", max(temp__))
              
            
       
            
        
        # transform the dictionary to a list of tuples and sort it by the score 
        result = sorted(score.items(), key=lambda tup: tup[1], reverse=True) 
        # return only the keys
        # result = [key for key, value in result]
        # retornamos los k documentos mas relevantes (de mayor similitud al query)
        print("Results retrieved")
        print("Top", k, "songs:")
        return result[:k]
    # FIXME 2: display the results in a dataframe
    def displayResults(self, result):
       
        track_ids = []
        track_name = []
        artists = []
        lyrics = []
        scores = []
        
        # open the file with the songs
        songs = pd.read_csv(self.original_file, delimiter=',', quotechar='"')
        for item in result:
            # info_new = "Rank:" + str(i)+ "Url :"+ dataton["url"][resuldato[0]]+ "score: "+ str(resuldato[1])
            #result_news.append(info_new)
            
            song = songs.iloc[item[0]- 1] 
            track_ids.append(song['track_id'])
            track_name.append(song['track_name'])
            artists.append(song['track_artist'])
            lyrics.append(song['lyrics'])
            scores.append(str(item[1]))
       
        df_news = pd.DataFrame({"ID": track_ids, "Track Name": track_name,"Artist": artists ,"Lyrics": lyrics, "Score": scores})
        return df_news

 # 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
# Test preprocess songs
index = InvertIndexSPIMI()
index.build(r".\Postgres\spotify_songs.csv")

In [48]:
# Test retrieval
# TODO: try gradio, display the results in a dataframe
# TODO: integrate with the gui of postgres
# TODO: delete comments, clean
index.retrieval("love", 5)

Index main ready
Length main ready
Query tfidf mas: 1.0
max1:  11618
0.45107389225285366   19.574652777169685
0.9073647704093012   21.083948516513868
0.17789160965896805   16.544933990278526
0.17576259404076   25.11801463185176
0.48974127509278426   30.04856940712638
0.39407875437667467   44.81142268967079
0.5355561817474288   24.73022005521399
0.04840794073981575   30.400022127064176
0.5346019975074117   19.268946497285125
0.07687760033682743   38.28429771919023
0.037616736368315216   39.12095018576755
0.08623731742305515   17.064566867227065
0.08186528644014697   17.975903262635285
0.12608663602006825   11.671359598980708
0.3053883677720277   19.275160745016468
0.049686481048943354   29.617763998348313
0.2631341563654293   22.370375476033793
0.12349078664201545   59.58349240587495
0.3788026318158344   15.539516846139245
0.18873625635635394   23.391411348715156
0.2297822777633704   25.617336270492803
0.11080029638106485   26.56315041892792
0.8499214651932182   22.50894098863475
0.0629

[(10738, np.float64(0.3788664755756362)),
 (10520, np.float64(0.24507085461672562)),
 (11316, np.float64(0.23774297219445306)),
 (11283, np.float64(0.23677603202718245)),
 (10790, np.float64(0.2324795123613357))]

In [54]:
index.displayResults(index.retrieval("Outkast", 10))

Index main ready
Length main ready
Query tfidf mas: 1.0
max1:  11449
0.06698737033925614   56.56244217926994
0.10003227897999077   37.87746615583372
0.07699179284458528   49.21263840684794
max:  11449
Results retrieved
Top 10 songs:


,ID,Track Name,Artist,Lyrics,Score
0,4UZlEcQmkAeh4mYokk8TNs,Let's Ride,Q-Tip,"NA To all the lowriders, and big-riders Lowrid...",0.10003227897999077
1,4V3N5LvUOh3yDPP16cwAhE,Aquemini,OutKast,"Even the sun goes down, heroes eventually die ...",0.07699179284458528
2,4GdB5M7GbwLZLouktYocFC,SpottieOttieDopaliscious,OutKast,"Damn, damn, damn, James Damn, damn, damn, Jame...",0.06698737033925614


In [14]:
print(index.length_main[6176])

13.312949504201775


In [5]:
xd = index.length_main.values()
countxd = np.unique(list(xd), return_counts=True)
print(countxd)

(array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.00000001, 1.00000001, 1.00000002, 1.00000006, 1.00000009,
       1.00000018, 1.0000007 , 1.00000141, 1.00000282, 1.00001127,
       1.00002254, 1.00004509, 1.0000902 , 1.00023594, 1.00047193,
       1.00072167, 1.00181143, 1.0028898 , 1.00578824, 1.01160939,
       1.02335357, 1.04725252, 1.09673785, 1.2028339 , 1.44680939,
       6.26120558]), array([10789,     1,     1,     2,     4,     1,     1,     2,     3,
           3,     1,     2,     1,     1,     1,     1,     1,     1,
           4,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1,     1,     1,     1,     1,     1,     1,
           1,     1,     1, 

### 2- Interfaz Gráfica

In [51]:
# TODO: refactor this function
def wrapper(query, k):
    result = index.retrieval(query, k)
    return index.displayResults(result)

In [52]:
# Blasterjaxx
print(wrapper("I really can't stay Baby it's cold outside I've got to go away Baby it's cold out there This evening has been Been hoping that you'd drop in So very nice I'll hold your hands, they're just like ice My mother will start to worry Beautiful, what's your hurry? My father will be pacing the floor Listen to that fireplace roar So really I'd better scurry Beautiful, please don't hurry Well maybe just a half a drink more Why don't you put some records on while I pour The neighbors might think Baby, it's bad out there Say, what's in this drink? No cabs to be had out there I wish I knew how Your eyes are like starlight To break the spell I'll take your hat, your hair looks swell I ought to say no, no, no, sir Mind if I move in closer? At least I'm gonna say that I tried What's the sense in hurting my pride? I really can't stay Baby don't hold out Baby it's cold outside I simply must go See that it's cold outside The answer is no I said it's cold out there This welcome has been How lucky that you dropped in So nice and warm Look out the window at that storm My sister will be suspicious Gosh, your lips look delicious My brother will be there at the door Waves upon a tropical shore My maiden aunt's mind is vicious Oh, your lips are delicious Maybe just a cigarette more Never such a blizzard before Hey I've got to go home Baby, you'll freeze out there Say, lend me your coat It's up to your knees out there You've really been grand I'm thrilled when you touch my hand But don't you see How can you do this thing to me? There's bound to be talk tomorrow Think of my life long sorrow At least there will be plenty implied If you caught pneumonia and died I really can't stay Get over that old lie Baby, baby it's cold outside", 5))

Index main ready
Length main ready
Query tfidf mas: 0.15198195891144
max1:  18277
1.6993091897933563   22.59436650910221
0.8495812467211721   20.227518656778884
7.25699842267689   21.926957334291153
3.3034667586372803   59.58349240587495
4.933777443347178   29.203757851352957
7.50170725008903   25.169804981412728
2.353067610462755   35.56346007658203
2.401056702285583   40.057314694553455
3.508796155840574   28.01129896322379
3.6192652311453335   45.64789082840359
4.043470621704924   35.895479293217434
2.948634182684044   46.24900416880392
14.829920609078123   27.043800276761132
3.8991137408882617   41.39355369474579
4.6645982244628375   52.66286108019149
4.419646920915704   32.334140505045404
3.8662046578602554   24.054152805987936
5.7833018543174175   20.282334430867117
3.915681706138361   40.57321671673982
1.7316463688881738   27.14964950848386
3.6478981987131474   31.386985198672946
4.248147902485245   28.185467006672944
3.88625939980927   19.737103858796793
3.6315960557340365   47

In [19]:
import gradio as gr

demo = gr.Interface(
    fn=wrapper,
    inputs=[gr.Text(label="Query"), gr.Slider(label="Top Scores")],
    outputs=[gr.Dataframe(
        headers=["ID", "Url", "Score"],
        datatype=["str", "str", "str"],
    )],
    title="Proyecto DB2",
    examples=[
        ["El pais de China y su cooperacion", 5],
        ["Salud y bienestar en la sociedad", 10],
    ],
    theme='HaleyCH/HaleyCH_Theme'
)



In [21]:
# demo.close()


Closing server running on port: 7860


In [20]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
